# Hyperparameter Tuning using HyperDrive

In [ ]:
import os
import csv

from sklearn import datasets
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.widgets import RunDetails

from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive import normal, choice

## Initialize Workspace and Experiment

In [ ]:
project_folder = './hyper_drive_proj'
ws = Workspace.from_config()
experiment_name = 'hyperDrive-experiment'
experiment=Experiment(ws, experiment_name)

## Dataset
Loading dataset is done inside train.py script.

## Creating Compute Target

In [ ]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

amlcompute_cluster_name = "demo-cluster"
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)

## Hyperdrive Configuration
Model params and configuration is explained in readme file.

In [ ]:
early_termination_policy = BanditPolicy(evaluation_interval=3, slack_factor=0.1, delay_evaluation=3)
param_sampling = RandomParameterSampling({
    'C': choice(0.01, 0.1, 1),
    '--max_iter': choice(0, 10, 50),
    'solver' : choice('liblinear','sag','lbfgs')
})

est = SKLearn(source_directory='.', compute_target=compute_target, entry_script = 'train.py', script_params={'--C':1.0, '--max_iter':100})

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(hyperparameter_sampling=param_sampling,
                                     primary_metric_name='normalized_root_mean_squared_error', 
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=2, 
                                     max_concurrent_runs=4, 
                                     policy=early_termination_policy,
                                     estimator=est)


In [ ]:
hd_run = exp.submit(hyperdrive_config)

## Run Details

In [ ]:
RunDetails(hyperdrive_run).show()

In [ ]:
hyperdrive_run.wait_for_completion(show_output=True)

## Best Model

In [ ]:
# get the best run and display the properties of the model
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run Id: ', best_run.id)
print('\n normalized_root_mean_squared_error:', best_run_metrics['normalized_root_mean_squared_error'])
print('\n Regularization Strength:',best_run_metrics['Regularization Strength:'])
print('\n Max iterations:',best_run_metrics['Max iterations:'])

In [ ]:
best_run

In [ ]:
import joblib
best_run.download_file(name='outputs/model.joblib', output_file_path='./outputs')
model = best_run.register_model(model_name='hyperdrive_run', model_path='outputs/model.joblib')

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

### NOTE: Since the AutoMl model is performing better and since we have to deploy only one model, skipping hyperDrive model deployment step as I already deployed Automl model.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

